# Mallows Kendall
### A python package to for Mallows Model with top-$k$ and complete rankings using Kendall's-$\tau$ distance
&emsp;&emsp; **By Ahmed Boujaada, Fabien Collas and Ekhine Irurozki**

We present methods for inference in the Mallows Model (MM), the best-known distribution for permutations. This is short tutorial for top-$k$ rankings and complete rankings under the Kendall's-$\tau$ distance. Theoretical details are given in

> Fabien Collas and Ekhine Irurozki (2020). Concentric mixtures of Mallows Models for top-k rankings: sampling and identifiability. In: International Conference on Machine Learning (ICML 21). 2021.

In [1]:
import numpy as np
import mallows_kendall as mk
import permutil as pu
import scipy as sp
import pandas as pd

# Kendall's-$\tau$ Distance

Let $\sigma$ and $\pi$ be two permutations of $n$ items. The Kendall's-$\tau$ distance $d(\sigma, \pi)$ counts the number of pairwise disagreements between $\sigma$ and $\pi$. When $\pi = e$, the Kendall's-$\tau$ distance between two rankings $\sigma$ and $\pi$ is equal to the number of inversions in the ranking $\sigma$. The Kendall's-$\tau$ distance between two permutations can be computed as follows: 

In [2]:
perm1 = np.array([3, 1, 2, 0, 4])
perm2 = np.array([3, 1, 4, 2, 0])
mk.distance(perm1, perm2)

6

The function builds upon Merge sort algorithm and runs in $O(n\log n)$. If only one permutation is given as input, it will be assumed that the second permutation is the identity permutation $e = (1, 2, \dots, n)$.

In [3]:
mk.distance(perm1)

5

The maximum value of the kendall's-$\tau$ distance between two permutations of length $n$ is $\frac{n(n-1)}{2}$. It is possible to get this value using the following function: 

In [4]:
n = 5
mk.max_dist(n)

10

For each $\sigma \in S_n$ there exists a unique vector, called inversion vector of $\sigma$ with regards to the Kendall's-$\tau$ distance, $V(\sigma) = (V_1(\sigma), \dotsc, V_{n-1}(\sigma))$ and such that $d(\sigma) = \sum_{j=1}^{n-1}V_j(\sigma)$.   
The $j^{th}$ element $V_j(\sigma)$ is given by: $V_j(\sigma) = \sum\limits_{i=j+1}^{n}\mathbb{1}_{\sigma(i) < \sigma(j)}$, $\forall j \in \{1, \dotsc, n-1\}$. It follows that $0 \le V_j(\sigma) \le n-j$. 

There is a bijection between each $\sigma \in S_n$ and each possible $V(\sigma)$ inversion vector. In the package the conversion from one form to another is done using the two following functions:  

- From $\sigma$ to $V(\sigma)$ (here $V$ is of the same length $n$ as the permutation, thus the final $n^{th}$ element of $V$ will always equal $0$) :

In [5]:
V_perm1 = mk.ranking_to_v(perm1)
V_perm1

array([3, 1, 1, 0, 0])

- From $V(\sigma)$ to $\sigma$ :

In [6]:
mk.v_to_ranking(V_perm1, n)

array([3, 1, 2, 0, 4])

In [7]:
mk.num_perms_at_dist(n=5)

array([[ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  2,  2,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  3,  5,  6,  5,  3,  1,  0,  0,  0,  0],
       [ 1,  4,  9, 15, 20, 22, 20, 15,  9,  4,  1]], dtype=uint64)

This package also allows to sample a permutation with a given number of inversions, i.e., at a given distance, where all the possible permutations have the same probability of being generated.

In [8]:
mk.sample_at_dist(n, dist= 4, sigma0=None)

array([1, 0, 4, 3, 2])

*note*: We point out that this notation is taken from Meilǎ, M., Phadnis, K., Patterson, A., & Bilmes, J. (2007). Consensus ranking under the exponential model. In *Proceedings of the 23rd Conference on Uncertainty in Artificial Intelligence, UAI 2007* (pp. 285–294). Note that the definition of the inversion vector difers from the one in the original paper of MM Fligner, M. A., & Verducci, J. S. (1986). Distance based ranking models. Journal of the Royal Statistical Society, 48(3), 359–369.
This different versions imply different expressions for the MM which will affect the capability of the MM to be used for partial permutations.

# Mallows Model (MM) for complete permutations
The probability mass function of a Mallows Model with central permutation $\sigma_0$ and dispersion parameter $\theta$ is $$p(\sigma) = \frac{exp(-\theta d(\sigma, \sigma_0))}{\psi},$$

and can be computed using the following function:

In [9]:
n = 5    
sigma = np.array([3,1,2,0,4])
sigma_0 = np.array(range(5))
theta = 0.1

mk.prob(sigma, sigma_0, theta)

0.008161793281714687

**Sampling** This package includes a sampler based on the factorization of the Kendall's-$\tau$  distance. In the later sections we will also present how this can be adapted to top-$k$  rankings. This differs to the classical sampling, usually done using the Repeated Insertion Model (RIM) and which can not be extended to top-$k$  rankings.

In [10]:
mk.sample(m=4,n=5,theta=1.5)

array([[2, 0, 3, 1, 4],
       [2, 0, 3, 1, 4],
       [0, 1, 2, 3, 4],
       [1, 3, 0, 2, 4]])

Note that in the package the sampling functions generates the samples considering $\sigma_0 = e$, identity permutation by default. But any other central permutation can be given as a parameter.  
In practice, we can draw a sample from a MM as follows:

In [11]:
mk.sample(m=4, n=5, theta=1.5, s0=np.array([4,3,2,1,0]))

array([[3, 4, 2, 1, 0],
       [3, 4, 2, 1, 0],
       [4, 3, 2, 1, 0],
       [4, 3, 1, 2, 0]])

In this package, we can specify also the parameter `phi` instead of `theta` when we consider the following equivalent expression for the Mallows Model, $$p(\sigma) \propto \phi^{d(\sigma,\sigma_0)}.$$ The relation between both expressions is given by $\phi = \exp(-\theta)$. This functionality holds for most functions. The sampling, for example, is done then as follows:

In [12]:
mk.sample(m=4,n=5,phi=.5)

array([[4, 0, 1, 2, 3],
       [1, 2, 0, 3, 4],
       [1, 0, 2, 3, 4],
       [0, 3, 2, 4, 1]])

**Expected distance** The expected value of Kendall's-$\tau$ distance under the MM is given by: 
$$\mathbb{E}[D] = \frac{n \cdot \exp(-\theta)}{1 - \exp(-\theta)} - \sum_{j=1}^n\frac{j \cdot \exp(-j \theta)}{1 - \exp(-j \theta)}$$

In [13]:
theta_mm = 0.7
expected_dist = mk.expected_dist_mm(n, theta_mm)
expected_dist

2.4578024409695693

**Variance** The variance of Kendall's-$\tau$ distance under the MM can be expressed as follows: 
$$\mathbb{V}[D] = \dfrac{ n \cdot \exp(-\theta) }{ (1 - \exp(-\theta))^2 } - \sum_{j=1}^n \dfrac{ j^2\exp(-j \theta) }{ (1 - \exp(-j \theta))^2 }$$

In [14]:
variance_dist = mk.variance_dist_mm(n, theta_mm)
variance_dist

2.763301117320907

In [15]:
sample_mm = mk.sample(m=4,n=5,phi=.9)

**Learning** Fitting a Mallows model Model is done in two stages. First, estimate the central permutation $\sigma_0$ and, second, compute the dispersion parameter. 

Function `median` uses Borda algorithm to approximate the central permutation

In [16]:
mk.median(sample_mm)

array([0, 3, 2, 4, 1])

The function `fit_mm` returns an approximation to the MLE of the parameters $\sigma_0$ and $\theta$. The consensus $\sigma_0$ is approximated with the well known Borda count algorithm. 

In [17]:
sigma_borda, phi_mm_mle = mk.fit_mm(sample_mm)
sigma_borda, phi_mm_mle 

(array([0, 3, 2, 4, 1]), 0.5943115810463766)

# Mallows Model for Top-$k$ rankings

**Sampling** In this part the sampling of the top-$k$ rankings is discussed.  
Perhaps the most natural idea to generate top-$k$ rankings would be to generate the full rankings, using the Repeated Insertion Model for example, and to cut these obtained permutations after position $k$. This is possible, yet it seems computationally non-optimal with a complexity of $O(n^2)$.    

Here we adapt the methods of previous sections to top-$k$ rankings. This method is based on sampling partially the inversion vectors. In practice in the package, a top-$k$ ranking can be sampled as follows: 

In [35]:
m = 15
n = 10 
k = 5
phi = 0.9 
sigma_0 = np.array(range(10))
sigma = np.random.permutation(n)
sample_top_k = mk.sample(m=m, n=n, k=k, phi=phi, s0=sigma_0)
sample_top_k

array([[ 0.,  3., nan, nan,  4., nan,  1., nan,  2., nan],
       [nan,  2.,  1.,  0., nan, nan,  4.,  3., nan, nan],
       [ 4.,  3.,  1.,  0., nan, nan, nan, nan, nan,  2.],
       [ 3.,  2., nan, nan,  4.,  0., nan, nan, nan,  1.],
       [ 0.,  3., nan, nan, nan, nan, nan,  4.,  1.,  2.],
       [ 1.,  4.,  2., nan, nan, nan, nan,  3., nan,  0.],
       [ 4.,  0., nan, nan, nan, nan,  3.,  1., nan,  2.],
       [nan,  3.,  1., nan, nan, nan,  0.,  4.,  2., nan],
       [ 3., nan, nan, nan,  2., nan,  0.,  1.,  4., nan],
       [nan,  0.,  3., nan, nan, nan,  1.,  4.,  2., nan],
       [ 2., nan, nan,  0.,  3.,  4.,  1., nan, nan, nan],
       [nan, nan, nan,  4.,  0., nan,  1.,  2.,  3., nan],
       [ 3., nan,  1.,  4., nan, nan, nan, nan,  0.,  2.],
       [ 3., nan, nan,  2., nan, nan,  0.,  4., nan,  1.],
       [ 0., nan, nan, nan, nan,  1., nan,  3.,  2.,  4.]])

**Distance** 

To compare top-$k$ rankings, we include an extension of the classical Kendall's-$\tau$ denoted as $p$-parametrized Kendall's-$\tau$ distance (Fagin et al., 2003). In practice we can use it, choosing the $p$ parameter in $[0,1]$, as follows: 

In [63]:
sample_top_k[0],sample_top_k[1]

(array([ 0.,  3., nan, nan,  4., nan,  1., nan,  2., nan]),
 array([nan,  2.,  1.,  0., nan, nan,  4.,  3., nan, nan]))

In [64]:
mk.p_distance(sample_top_k[0], sample_top_k[1], k=k, p=0)

18

In [65]:
mk.p_distance(sample_top_k[0], sample_top_k[1], k=k, p=1)

24


**Expected distance** The expected value of Kendall's-$\tau$ distance for top-$k$ rankings under the MM is given by: 
$$\mathbb{E}[D] = \frac{k \cdot \exp(-\theta)}{1 - \exp(-\theta)} - \sum_{j=n-k+1}^n\frac{j \cdot \exp(-j \theta)}{1 - \exp(-j \theta)}$$

In [37]:
expected_dist = mk.expected_dist_top_k(n,k,theta_mm)
expected_dist

4.732931959531709

**Variance** The variance of Kendall's-$\tau$ distance for top-$k$ rankings under the MM can be expressed as follows: 
$$\mathbb{V}[D] = \dfrac{ k \cdot \exp(-\theta) }{ (1 - \exp(-\theta))^2 } - \sum_{j=n-k+1}^n \dfrac{ j^2\exp(-j \theta) }{ (1 - \exp(-j \theta))^2 }$$

In [38]:
variance_dist = mk.variance_dist_top_k(n,k,theta_mm)
variance_dist

8.391580684957216

# Generalized Mallows Model (GMM) for complete permutations

The Generalized Mallows Model (GMM) is an extension of the Mallows Model, for which there are $n-1$ dispersion parameters $\theta_j$ ($1 \le j < n$), each affecting a particular position of the permutation.   
Formally, the GMM under Kendall's-$\tau$ distance is expressed as follows: 

$$p(\sigma)=\dfrac{\exp(\sum_{j=1}^{n-1}-\theta_j V_j(\sigma\sigma_0^{-1}))}{\psi(\theta)}$$
where $\psi(\theta) = \prod_{j=1}^{n-1} \psi_j(\theta_j) = \prod_{j=1}^{n-1} \frac{1-\exp(-\theta_j(n-j+1))}{1 - \exp(-\theta_j)}$

In [39]:
def prob_GMM(sigma, sigma_0, theta):
    n = len(sigma)
    sigma_0_inv = pu.inverse(sigma_0)
    V = mk.ranking_to_v(pu.compose(sigma, sigma_0_inv))
    psi = np.prod(np.array([(1 - np.exp(( - n + j ) * theta[j]))/(1 - np.exp(-theta[j])) for j in range(n-1)]))
    return np.exp( np.sum ( [ -theta[j] * V[j] for j in range(n-1) ] ) ) / psi 
    
sigma = np.array([3,1,2,0,4])
sigma_0 = np.array(range(5))
theta = [0.5,0.2,0.6,0.3]

print(prob_GMM(sigma, sigma_0, theta))

0.00439275005307042


**Sampling** It is also possible to sample the Generalized Mallows Models (GMM), using the same process as for the MM only with a change in the probability of $V_j$ as follows:

$$p(V_j(\sigma\sigma_0^{-1}) = r) = \frac{\exp(-\theta r)}{\psi_j(\theta)}, \; \forall r \in {0, \dotsc, n-j}$$

In the package it is quite similar as for the classical Mallows Models, only that the dispersion parameter will be given as a list. 

In [40]:
m = 4000
n = 5
theta_gmm = [0.5,0.2,0.6,0.3]
identity = np.array(range(n))
sample_gmm = np.array(mk.sample(m, n, theta = theta_gmm))

The expected value of each term $V_j(\sigma)$, where $\sigma$ is a random Mallows permutation, is expressed as follows:
$$ \mathbb{E}[V_j] = \frac{\exp(-\theta_j)}{1 - \exp(-\theta_j)} - \frac{(n-j+1)\exp(-\theta_j(n-j+1))}{1 - \exp(-\theta_j(n-j+1))}, \;\forall j \in \{1, \dotsc, n-1\} $$
And the values are computed as follows:

In [41]:
expected_v_gmm = mk.expected_v(n, theta_gmm)
print(expected_v_gmm)

[1.09436663 1.25279068 0.62226834 0.42555748]


The expected value of Kendall's-$\tau$ distance under the GMM is then given by: 
$$\mathbb{E}[D] = \sum_{j=1}^{n-1} \mathbb{E}[V_j]$$

In [42]:
expected_dist_gmm = np.sum(expected_v_gmm)
print(expected_dist_gmm)

3.394983134634349


The variance of each term $V_j(\sigma)$ under the GMM can be expressed as follows: 
$$\mathbb{V}[V_j] = \dfrac{ \exp(-\theta_j) }{ (1 - \exp(-\theta_j))^2 } - \sum_{j=1}^n \frac{ (n-j+1)^2 \exp(-(n-j+1) \theta_j) }{ (1 - \exp(-(n-j+1) \theta_j))^2 }, \;\forall j \in \{1, \dotsc, n-1\} $$

In [43]:
variance_v_gmm = mk.variance_v(n, theta_gmm)
print(variance_v_gmm)

[1.48213789 1.20855956 0.56066479 0.24445831]


The expected value of Kendall's-$\tau$ distance under the GMM is then given by: 
$$\mathbb{V}[D] = \sum_{j=1}^{n-1} \mathbb{V}[V_j]$$

In [44]:
variance_dist_gmm = np.sum(variance_v_gmm)
print(variance_dist_gmm)

3.495820543795291


**Learning** An approximation to the MLE for $\theta$ and $\sigma_0$ given a sample of i.i.d Mallows permutations is as follows: 

In [45]:
sigma_borda, theta_gmm_mle = mk.fit_gmm(sample_gmm)
print(sigma_borda)
print(theta_gmm_mle)

[0 2 1 3 4]
[0.5109401970612621, 0.19755236580605012, 0.616607960418165, 0.2757338026671021]
